In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import tool

以上为导入环境的步骤

In [38]:
# --- LLM API 配置 ---
LLM_CONFIG = {
    # 替换为你的 API Key
    "api_key": "sk-stgjqeuzoevxkohlssoiqcpjdydttorxsccrhnfhlmrflawh",
    # 替换为你的 API 接入点 (Base URL)
    # 例如 DeepSeek: 'https://api.deepseek.com'
    "base_url": "https://api.siliconflow.cn/v1/",
    # 替换为你需要使用的模型名称
    "model_name": "zai-org/GLM-4.6",
}
LLM_CONFIG = {
    # 替换为你的 API Key
    "api_key": "sk-PJfHRIM2Q0K8vXajt7pBjrXFhxbVP3GSsqw6KItdSM2MvOf4",
    # 替换为你的 API 接入点 (Base URL)
    # 例如 DeepSeek: 'https://api.deepseek.com'
    "base_url": "https://api.moonshot.cn/v1",
    # 替换为你需要使用的模型名称
    "model_name": "kimi-k2-turbo-preview",
}


llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
response=llm.invoke("你是什么模型")
print(response)


content='我是由**Moonshot AI**训练的大语言模型，名字叫**Kimi**。我的知识截止时间是**2025年4月**，擅长用自然流畅的语言和你互动交流。有什么我可以帮你的吗？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 10, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'kimi-k2-turbo-preview', 'system_fingerprint': None, 'id': 'chatcmpl-692562846bdbe892a1e11729', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--b36ad6b8-2b73-460c-bb56-efae7fe69cd8-0' usage_metadata={'input_tokens': 10, 'output_tokens': 43, 'total_tokens': 53, 'input_token_details': {}, 'output_token_details': {}}


这个为新建的llm的接口，通过这个接口我们能够调用我们的模型来完成各种各样的任务。

In [ ]:
llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
response=llm.invoke("这是第几次调用？")
print(response.content)
response=llm.invoke("这是第几次调用？")
print(response.content)
response=llm.invoke("这是几点了？")
print(response.content)

这是**第一次**调用。
这是**第一次**调用。
我无法直接获取实时时间，但你可以查看你设备上的时间（手机、电脑或手表），那就是当前的准确时间。


现在我们尝试来建立一个agent,langchain的agent拥有可以不断思考然后再执行的能力,可以给agent配置各种各样的工具和记忆功能，来让agent完成复杂的任务。

我们首先来给agent配置一个记忆的能力。

In [48]:
llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm。快速的回答用户。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "这是第几次调用？"}]},
                    config={
                    "configurable": {
                        "thread_id": 1,
                        "max_iterations": 10,
                    },
                    "recursion_limit": 20,
                },)
last_message=response["messages"][-1].content
print(last_message)
response=agent.invoke({"messages": [{"role": "user", "content": "这是第几次调用？"}]},
                    config={
                    "configurable": {
                        "thread_id": 1,
                        "max_iterations": 10,
                    },
                    "recursion_limit": 20,
                },)
last_message=response["messages"][-1].content
print(last_message)
response=agent.invoke({"messages": [{"role": "user", "content": "现在是几点钟？"}]},
                    config={
                    "configurable": {
                        "thread_id": 1,
                        "max_iterations": 10,
                    },
                    "recursion_limit": 20,
                },)
last_message=response["messages"][-1].content
print(last_message)

这是第 1 次调用。
这是第 2 次调用。
现在是 2025-06-25 14:42（北京时间）。


这样就让大模型拥有了记忆的功能。

我们发现，langchain创建的agent不能获取到当前的时间，这是我们调用的实际上是最基础的模型，最基础的模式是不具备知道时间的功能的，因此我们需要给agent添加一个获取当前时间的工具。

In [ ]:
import datetime

@tool
def get_time()->str:
    '''用户想要知道时间的时候使用这个工具'''
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return current_time


llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm。快速的回答用户。最后回答要包含完整文案。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            tools=[get_time],
            # checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "现在是几点钟？"}]},)
last_message=response["messages"][-1].content
print(last_message)


现在是2025年11月25日 下午4点10分21秒。


现在agent就具备一定的能力了。

In [ ]:
@tool
def clauculate(a:int,b:int)->str:
    '''有数字比大小的时候一定要使用这个工具'''
    if a>b:
        return f"{a}大于{b}"
    elif a<b:
        return f"{a}小于{b}"
    else:
        return f"{a}等于{b}"


llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm,注意工具的调用。快速的回答用户。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            tools=[clauculate],
            # checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "3.11和3.9哪个更大？"}]},)
last_message=response["messages"][-1].content
print(last_message)

3.9比3.11更大。

这是因为虽然两个数的整数部分都是3，但比较小数部分时，0.9（也就是9/10）大于0.11（也就是11/100）。


In [55]:
llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm,注意工具的调用。快速的回答用户。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            # checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "今天ai有什么新闻？"}]},)
last_message=response["messages"][-1].content
print(last_message)


我来帮你查一下今天 AI 领域的最新新闻。请稍等几秒钟。


In [54]:
from langchain_community.tools import DuckDuckGoSearchResults

llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm,注意工具的调用。快速的回答用户。"
agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            tools=[DuckDuckGoSearchResults()],
            # checkpointer=InMemorySaver(),
        )
response=agent.invoke({"messages": [{"role": "user", "content": "最新的ai有什么新闻?想了解最新的谷歌的模型"}]},)
last_message=response["messages"][-1].content
print(last_message)


根据最新搜索结果，以下是关于谷歌AI模型的重要新闻：

## 谷歌Gemini 2.0 Flash Thinking 模型

**重大发布：**
谷歌最近发布了 **Gemini 2.0 Flash Thinking** 实验性AI模型，这是谷歌在推理AI领域的重要突破。

**主要特点：**
1. **推理能力增强** - 这个模型能够像人类一样进行思考，并在回答复杂问题时展示其推理过程
2. **透明度提升** - 用户可以清楚地看到AI是如何一步步得出答案的
3. **复杂问题解决** - 特别适用于处理困难的代码和数学问题
4. **多模态能力** - 能够理解和处理多种类型的信息

**技术亮点：**
- 这是谷歌对OpenAI o1-preview和o1-mini模型的回应
- 模型经过专门训练，能够生成"思考过程"作为回应的一部分
- 已经集成到Gemini应用中，用户可以直接体验

**应用前景：**
- 更便宜地访问AI功能，为开发者提供成本效益
- 能够使用其他谷歌应用如YouTube等
- 在教育、编程、数学等领域有重要应用价值

这个发布标志着谷歌在AI推理能力方面迈出了重要一步，与OpenAI展开了更激烈的竞争。
